In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import SimpleITK as sitk

from src.evaluation.scores import dice, dice2d
from src.resampling.utils import get_np_volume_from_sitk
from src.resampling.resampling import resample_np_binary_volume

In [ ]:
#!python ../src/niftynet/rename_output.py ../data/segmentation_output/ ../data/segmentation_output_renamed

In [ ]:
prediction_folder = '../data/segmentation_output_renamed'
groundtruth_folder = '../data/resampled/'
bb_filepath = '../data/bbox.csv'

In [ ]:
prediction_files = [
            f
            for f in Path(prediction_folder).rglob('*.nii.gz')
        ]
prediction_files.sort(key=lambda x: x.name.split('_')[0])   
patient_name_predictions = [
    f.name.split('.')[0] for f in prediction_files
]



In [ ]:

groundtruth_files = [
    f for f in Path(groundtruth_folder).rglob('*gtvt.nii.gz') if f.name.split('_')[0] in patient_name_predictions
]
groundtruth_files.sort(key=lambda x: x.name.split('_')[0])

In [ ]:
bb_df = pd.read_csv(bb_filepath).set_index('PatientID')

In [ ]:
results_df = pd.DataFrame(columns=['PatientID', 'dice3D', 'dice2D'])
for i, f in enumerate(prediction_files):
    gt_file = groundtruth_files[i]
    patient_name = f.name.split('.')[0]
    print('Evaluating patient {}'.format(patient_name))
    bb = (bb_df.loc[patient_name, 'x1'], bb_df.loc[patient_name, 'y1'], bb_df.loc[patient_name, 'z1'],
          bb_df.loc[patient_name, 'x2'], bb_df.loc[patient_name, 'y2'], bb_df.loc[patient_name, 'z2'])
    sitk_pred = sitk.ReadImage(str(f.resolve()))
    sitk_gt = sitk.ReadImage(str(gt_file.resolve()))
    np_pred, px_spacing_pred, origin_pred = get_np_volume_from_sitk(sitk_pred)
    np_gt, px_spacing_gt, origin_gt = get_np_volume_from_sitk(sitk_gt)
    np_pred = resample_np_binary_volume(np_pred, origin_pred, px_spacing_pred, px_spacing_gt, bb)
    np_gt = resample_np_binary_volume(np_gt, origin_gt, px_spacing_gt, px_spacing_gt, bb)
    results_df = results_df.append({'PatientID': patient_name, 'dice3D': dice(np_gt, np_pred), 
                                    'dice2D':dice(np_gt, np_pred)}, ignore_index=True)

    
    


In [ ]:
results_df